In [1]:
import geopandas as gpd
from cartoframes.auth import set_default_credentials 

set_default_credentials("/Users/juanluis/Documents/credentials/carto_creds.json")


In [1050]:
import numpy as np, pandas as pd
import freetype
from shapely.geometry import *
from shapely.ops import unary_union, substring
from shapely import affinity as aff
from cartoframes.viz import Layer, Layout, Map, basemaps

face = freetype.Face("fonts/Vera.ttf") # path to your font
face.set_char_size(3000, hres = 100)
# These parameters are ok for Vera, but should be adjusted for each font.
char_width = 1500
space_between_chars_width = int(char_width*1/3)

get_width, get_height = (lambda b : b[2] - b[0], lambda b : b[3] - b[1])

def char_to_geometry(char, acc_offset):
    face.load_char(char)
    outline = face.glyph.outline
    
    X,Y = np.array(outline.points).T
    X = X - X.min() + acc_offset
    points = [(x,y) for x,y in zip(X,Y)]
    lines = []
    start = 0
    for end in outline.contours: # we travel each contour and generate a LineString
        points_in_line = points[start:end + 1] + [points[start]]
        l = LineString(points_in_line)
        lines.append(l)
        start = end + 1
    bbox = [X.min(), Y.min(), X.max(), Y.max()] # we compute the bounding box
    return lines, bbox
    
def text_as_geometry(text, coords, scale_factor = 5, rotate_angle = 0):
    n = len(text)
    char_geometries = []
    lat, lon = coords
    acc_offset = 0
    for char in text:
        try:
            m, bbox = char_to_geometry(char, acc_offset)
            offset = get_width(bbox)
            acc_offset += offset + space_between_chars_width
            char_geometries.extend(m)
        except:
            acc_offset += char_width
    ml = MultiLineString(char_geometries)
    Bbox = ml.bounds
    w = get_width(Bbox)
    h = get_height(Bbox)
    f = max(w,h)
    ml = aff.translate(ml, -Bbox[2] + w/2,  -Bbox[3] + h/2)
    ml = aff.scale(ml, scale_factor/(f*100), scale_factor/(f*100), origin = (lon, lat))
    ml = aff.rotate(ml, rotate_angle)
    return ml

word = "What's Up!"
lat, lon =  37.771757, -99.618516

geom = text_as_geometry(word, (lat, lon), scale_factor = 650, rotate_angle = 5)
df = pd.DataFrame([ {"geom" : geom}])
Layer(df, geom_col = "geom")

In [71]:
from cartoframes.viz import Map
Map([l]).publish("whats_up_usa", password=None)

{'id': '59b9b0b8-91cf-4715-be37-c11bceeb50dd',
 'url': 'https://team.carto.com/u/juanluisr/kuviz/59b9b0b8-91cf-4715-be37-c11bceeb50dd',
 'name': 'whats_up_usa',
 'privacy': 'public'}

In [1349]:
available_chars = [chr(char[0]) for char in face.get_chars()]
"".join(available_chars)

'\x00\x1d !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz\xa0\xadß;ЁЂЃЄЅІЇЈЉЊЋЌЍЎЏАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ\x00'

In [1358]:
face = freetype.Face("fonts/ChalkboardSE.ttc") # path to your font

In [1346]:
text = "How much wood could a woodchuck chuck if a woodchuck could chuck wood? As much wood as a woodchuck could chuck, if a woodchuck could chuck wood>>>>>>>>>>>>>>>>>>>"



In [1359]:
from math import atan2, degrees, radians, sin, cos
def fit_text_as_geometry_between_two_points(text, A, B):
    mid_coords = ((B[1] + A[1])/2,  (B[0] + A[0])/2)
    angle = atan2(B[1] - A[1],  B[0] - A[0])
    bbox = LineString([A,B]).bounds
    h = get_height(bbox)
    w = get_width(bbox)
    scale_factor = 100*(abs(h*sin(angle)) + abs(w*cos(angle)))
    ml = text_as_geometry(text, mid_coords, scale_factor, degrees(angle))
    return ml

In [1411]:
n = len(text) + 1
angle = np.linspace(0,1.5*2*np.pi, n)
radius = np.linspace(0.1, 0.5,n) * 10

X = radius * np.cos(angle) + lon
Y = radius * np.sin(angle) + lat
points = [(x,y) for x,y in zip(X,Y)]

char_list = []
for char, a,b in zip(text, points,points[1:]):
    if char == " ":
        continue
    try:
        char_l = fit_text_as_geometry_between_two_points(char, a, b)
        char_list.extend([*char_l])
    except Exception as e:
        print(char, e)

text_ml = MultiLineString(char_list)
text_gdf = gpd.GeoDataFrame(geometry=[text_ml])
spiral_l = Layer(text_gdf, style = basic_style(color = "orange"))

/var/folders/cx/865gkv912xj3_mfvrtptff_r0000gp/T/ipykernel_85657/1756920780.py:15: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  char_list.extend([*char_l])
/var/folders/cx/865gkv912xj3_mfvrtptff_r0000gp/T/ipykernel_85657/1756920780.py:15: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  char_list.extend([*char_l])


In [1041]:
import wikipedia
barcelona = wikipedia.page("Barcelona")

In [83]:
barcelona_text = barcelona.summary

In [86]:
words = (barcelona_text
 .replace("\n", " ")
 .split(" ")
)

In [843]:
import string
accepted_chars = string.ascii_lowercase + "ñ()1234567890"

In [1362]:
word_is_valid = lambda word: not len(set(word.lower()) - set(accepted_chars)) > 1
filtered_words = list(filter(word_is_valid, words))

In [1363]:
text = " ".join(filtered_words)

In [1364]:
def split_by_nth(text, n):
    return [text[:n]] + split_by_nth(text[n:], n) if len(text) > n else [text]

In [1372]:
segments = split_by_nth(text, 9)
segments = segments[:64]

In [1373]:
from hilbertcurve.hilbertcurve import HilbertCurve
p = len(segments)
hilbert_curve = HilbertCurve(p, 2)
distances = list(range(p))
points = np.array(hilbert_curve.points_from_distances(distances))

In [1374]:
mid_coords = box(*LineString(points).bounds).centroid
barcelona_coords = (2.196105,41.398103)


In [1400]:
segment_list = []
for segment, a, b in zip(segments, points, points[1:]):
    d = degrees(atan2(b[1] - a[1],  b[0] - a[0]))
    if int(abs(d)) == 90:
        segment = " " + segment + " "
    segment_l = fit_text_as_geometry_between_two_points(segment, a, b)
    segment_list.extend([*segment_l])

/var/folders/cx/865gkv912xj3_mfvrtptff_r0000gp/T/ipykernel_85657/3442972639.py:7: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  segment_list.extend([*segment_l])
/var/folders/cx/865gkv912xj3_mfvrtptff_r0000gp/T/ipykernel_85657/3442972639.py:7: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  segment_list.extend([*segment_l])


In [1394]:
def transform_geometry(g, scale_by = 1, rotate_by = 0, center_at= None):
    if center_at:
        c = box(*g.bounds).centroid
        g = aff.translate(g, center_at[0] - c.x, center_at[1] - c.y)
    g = aff.scale(g, scale_by, scale_by, origin = "center")
    g = aff.rotate(g, rotate_by)
    return g

In [1401]:
hilbert_ml = MultiLineString(segment_list)
hilbert_ml = transform_geometry(hilbert_ml, 0.001, 0, barcelona_coords)

In [1396]:
from cartoframes.viz import Map, basic_style

In [1402]:
face.set_char_size(3000, hres = 100)
# These parameters are ok for Vera, but should be adjusted for each font.
char_width = 1500
space_between_chars_width = int(char_width*1/3)

In [1410]:
hilbert_gdf = gpd.GeoDataFrame(geometry=[hilbert_ml])
hilbert_l = Layer(hilbert_gdf,  style = basic_style(color = "orange"), encode_data = False)


In [1404]:
layout = Layout([
Map(spiral_l, basemap = basemaps.darkmatter), Map(hilbert_l,  basemap = basemaps.darkmatter),
], map_height = 400)

In [1405]:
layout

In [1406]:
layout.publish("text_to_geom_follow_line", None, if_exists="replace")

{'id': '79946422-c2f8-4836-b813-1f2c4b55349f',
 'url': 'https://team.carto.com/u/juanluisr/kuviz/79946422-c2f8-4836-b813-1f2c4b55349f',
 'name': 'text_to_geom_follow_line',
 'privacy': 'public'}

# Thames River

In [5]:
from google.cloud import bigquery
from dotenv import load_dotenv
load_dotenv("/Users/juanluis/Documents/credentials/env.txt")

bq_client = bigquery.Client()

In [33]:
q = """
SELECT  geometry
FROM `bigquery-public-data.geo_openstreetmap.planet_relations`
WHERE id = 2263653
"""
thames_df = bq_client.query(q).result().to_geodataframe()

In [37]:
g = thames_df.iloc[0].geometry

In [40]:
thames = g[0]

/var/folders/cx/865gkv912xj3_mfvrtptff_r0000gp/T/ipykernel_3022/2499522335.py:1: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  thames = g[0]


In [41]:
df = pd.DataFrame([ {"geom" : thames}])
Layer(df, geom_col = "geom")

# Europe's borders

In [5]:
bq_client = bigquery.Client()
europe_gdf = bq_client.query("SELECT * FROM `carto-do-public-data.eurostat.geography_glo_nuts0_2021`").result().to_geodataframe()

In [60]:
geom = europe_gdf.iloc[7].geom

In [87]:
from tqdm import tqdm
import wikipedia, iso3166


In [182]:
def get_longest_linestring_from_polygon(polygon):
    "Given a polygon, returns the longest linestring which defines its boundary"
    if not isinstance(polygon.boundary, LineString):
        lines = list(polygon.boundary.geoms)
        max_l = lines[0]
        for l in lines[1:]:
            if l.length > max_l.length:
                max_l = l
    else:
        max_l = polygon.boundary
    return max_l

def get_country_name(alpha2):
    try:
        return iso3166.countries_by_alpha2.get(alpha2).apolitical_name
    except:
        if alpha2 == "UK":
            return "United Kingdom" #get_country_name("GB")
        elif alpha2 == "EL":
            return "Greece" #get_country_name("GR")
    

In [1441]:
def map_text_to_points(text, points):
    """Maps each character along the trajectory defined by points.
    The length of the linestring is the minimum between the number of chars and the number of points -1"""
    char_list = []
    for char, a,b in zip(text, points,points[1:]):
        if char == " ":
            continue
        char_l = fit_text_as_geometry_between_two_points(char, a, b)
        char_list.extend([*char_l])
    return MultiLineString(char_list)

In [138]:
europe_gdf["country_name"] = europe_gdf.CNTR_CODE.apply(get_country_name)

In [ ]:
wikipedia_country = {}

In [145]:
for country in tqdm(europe_gdf.country_name.values):
    if country not in wikipedia_country:
        try:
            p = wikipedia.page(f"{country} country")
            wikipedia_country[country] = p.content
        except:
            print(country)

 22%|████████████████████████████▎                                                                                                      | 8/37 [00:00<00:02, 12.24it/s]

Czechia


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:12<00:00,  2.97it/s]


In [147]:
wikipedia_country["Czechia"] = wikipedia.page("Czech Republic").content

In [253]:
char_distance = 0.1

linestring_country = {}

for country, geom in tqdm(europe_gdf[["country_name", "geom"]].values):
    buffered_geom = geom.buffer(-char_distance)
    if buffered_geom.area > 0:
        line = get_longest_linestring_from_polygon(buffered_geom)
        linestring_country[country] = line
    
text_geometries = {}
for country in linestring_country:
    line = linestring_country.get(country)
    text = wikipedia_country.get(country)
    line = line.simplify(char_distance)
    distances = np.arange(0, line.length, char_distance)
    points = [list(line.interpolate(distance).coords)[0] for distance in distances]    
    text_ml = map_text_to_points(text, points)
    text_geometries[country] = text_ml

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:02<00:00, 15.39it/s]
/var/folders/cx/865gkv912xj3_mfvrtptff_r0000gp/T/ipykernel_83318/803002423.py:9: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  char_list.extend([*char_l])
/var/folders/cx/865gkv912xj3_mfvrtptff_r0000gp/T/ipykernel_83318/803002423.py:9: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  char_list.extend([*char_l])


In [254]:
gdf = gpd.GeoDataFrame(index = list(text_geometries.keys()), geometry = list(text_geometries.values()))

## El Cano trip

In [1055]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
elcano = gpd.read_file('data/viaje_elcano.kml', driver='KML')

In [1432]:
survivors =\
"""Juan Sebastián Elcano, de Guetaria;
Francisco Albo, de Axio;
Miguel de Rodas, de Rodas;
Juan de Acurio, de Bermeo;
Antonio Lombardo (Pigafetta), de Vicenza;
Martín de Yudícibus, de Savona;
Hernando de Bustamante, de Mérida;
Nicolás el Griego, de Nauplia;
Miguel Sánchez de Rodas, de Rodas;
Antonio Hernández Colmenero, de Ayamonte;
Francisco Rodríguez, portugués de Sevilla;
Juan Rodríguez, de Huelva;
Diego Carmena Gallego, de Bayona;
Hans, de Aquisgrán;
Juan de Arratia, de Bilbao;
Vasco Gómez Gallego el Portugués, de Bayona;
Juan de Santander, de Cueto;
Juan de Zubileta, de Baracaldo"""

In [1434]:
survivors_str =  " - ".join(survivors.split("\n"))

In [1307]:
drop_z_ls = lambda geom : LineString([xy[0:2] for xy in list(geom.coords)])
reverse_ls = lambda geom : substring(geom, geom.length, 0)


In [1308]:
sevilla_brazil, long_journey = list(map(drop_z, elcano.head(2).geometry.values))
brazil_sevilla = reverse_ls(sevilla_brazil)

journey = LineString([p for l in [long_journey, brazil_sevilla] for p in l.coords])

In [1309]:
points = np.array(list(journey.coords))
longitudes = points.T[0]
crossing_antimeridian = np.argmax(longitudes - np.roll(longitudes, 1))

In [1310]:
new_points = \
np.vstack(
    [points[crossing_antimeridian:],
     points[:crossing_antimeridian],]
)

In [1311]:
fixed_journey = reverse_ls(LineString([(p[0],p[1]) for p in new_points]))

In [1312]:
char_distance = 0.5

In [1313]:
distances = np.arange(0, fixed_journey.length, char_distance)
fixed_journey_points = [list(fixed_journey.interpolate(distance).coords)[0] for distance in range(len(tripulacion_str))]  

In [1413]:
face = freetype.Face("fonts/Savoye LET.ttc") # path to your font

In [1436]:
tripulacion_journey_geom = map_text_to_points(survivors_str, fixed_journey_points)

/var/folders/cx/865gkv912xj3_mfvrtptff_r0000gp/T/ipykernel_85657/803002423.py:9: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  char_list.extend([*char_l])
/var/folders/cx/865gkv912xj3_mfvrtptff_r0000gp/T/ipykernel_85657/803002423.py:9: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  char_list.extend([*char_l])


In [1437]:
df = pd.DataFrame([ {"geom" : tripulacion_journey_geom}])
elcano_trip = Layer(df, style=basic_style(color = "brown"), geom_col = "geom")

In [1439]:
trip_el_cano_map = Map([elcano_trip], basemap=basemaps.voyager)
trip_el_cano_map

In [1440]:
trip_el_cano_map.publish("elcano_trip_around_the_world", None, if_exists="replace")

{'id': 'f50741f6-798d-468a-a8c5-cb0e4341bb5d',
 'url': 'https://team.carto.com/u/juanluisr/kuviz/f50741f6-798d-468a-a8c5-cb0e4341bb5d',
 'name': 'elcano_trip_around_the_world',
 'privacy': 'public'}

# Borrador


In [845]:
def split_word(word, max_length = 10):
    if len(word) == 0:
        return []
    else:
        return [word[:max_length]] + split_word(word[max_length:], max_length)

In [859]:
split_words = []
for word in filtered_words:
    split_words.extend(split_word(word, 6))

In [860]:
max_segment_length = 8
fill = " "
segmented_words = []
current_word = ""
for next_word in split_words:
    if len(current_word) + len(next_word) + 1 < max_segment_length:
        current_word += f"{fill}{next_word}"
    else:
        chars_missing = max_segment_length - len(current_word)
        n, r = divmod(chars_missing,2)
        head, tail = n*fill, (n + r)*fill
        segment = f"{head}{current_word}{tail}"
        segmented_words.append(segment)
        current_word = next_word
    

In [861]:
n_segments_desired = 256
segmented_words = segmented_words + segmented_words[:n_segments_desired - len(segmented_words)]
segmented_words = segmented_words[:n_segments_desired]

In [ ]:
import utm
zone_number = utm.latlon_to_zone_number(barcelona_coords[1], barcelona_coords[0])
zone_letter = utm.latitude_to_zone_letter(barcelona_coords[0])
#new_points = np.array(utm.to_latlon(points.T[0], points.T[1], zone_number, zone_letter = zone_letter)).T